# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [61]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [62]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san antonio de pale,-1.4014,5.6325,25.30,82,99,5.71,GQ,1729891520
1,1,port-aux-francais,-49.3500,70.2167,2.25,67,58,10.73,TF,1729891521
2,2,waitangi,-43.9535,-176.5597,14.34,98,92,1.34,NZ,1729891522
3,3,mandali,33.7499,45.5541,17.49,22,0,3.44,IQ,1729891523
4,4,takoradi,4.8845,-1.7554,25.74,91,100,3.88,GH,1729891227


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [95]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
   'Lng', 
    'Lat',                            
    geo=True,                                
    tiles='OSM',                             
    size='Humidity',                         
    color='City',                           
    legend='right',                          
    frame_width=800,                         
    frame_height=500,                        
    fill_alpha=0.4,                          
    title='City Locations with Humidity Represented by Point Size',
    xlabel='x',                     
    ylabel='y',

)



# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) &  
    (city_data_df['Humidity'] < 60) &                                      
    (city_data_df['Cloudiness'] < 20)                                      
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(20)  

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,afif,23.9065,42.9172,21.66,52,18,6.94,SA,1729891615
92,92,hoopstad,-27.8327,25.9083,22.15,12,0,3.31,ZA,1729891626
97,97,lompoc,34.6391,-120.4579,24.52,47,11,2.24,US,1729891631
125,125,murzuq,25.9155,13.9184,20.23,31,5,4.32,LY,1729891665
150,150,port hedland,-20.3167,118.5667,24.03,57,4,2.57,AU,1729891692
206,206,crane,31.3974,-102.3501,26.19,27,5,4.20,US,1729891757
208,208,guerrero negro,27.9769,-114.0611,24.37,44,6,6.00,MX,1729891759
215,215,santa sylvina,-27.8326,-61.1375,26.14,38,2,9.50,AR,1729891769
233,233,aoulef,26.9667,1.0833,24.93,33,0,4.20,DZ,1729891790
234,234,rustam,27.9690,68.8048,28.82,21,0,1.52,PK,1729891791


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(20) 

,City,Country,Lat,Lng,Humidity,Hotel Name
82,afif,SA,23.9065,42.9172,52,
92,hoopstad,ZA,-27.8327,25.9083,12,
97,lompoc,US,34.6391,-120.4579,47,
125,murzuq,LY,25.9155,13.9184,31,
150,port hedland,AU,-20.3167,118.5667,57,
206,crane,US,31.3974,-102.3501,27,
208,guerrero negro,MX,27.9769,-114.0611,44,
215,santa sylvina,AR,-27.8326,-61.1375,38,
233,aoulef,DZ,26.9667,1.0833,33,
234,rustam,PK,27.9690,68.8048,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [76]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel", 
    "limit": 1,                           
    "apiKey": geoapify_key                
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lon = row['Lat'], row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
afif - nearest hotel: Samaya Royal Afif
hoopstad - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
murzuq - nearest hotel: فندق باريس - مرزق
port hedland - nearest hotel: The Esplanade Hotel
crane - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
santa sylvina - nearest hotel: No hotel found
aoulef - nearest hotel: دار الضياف
rustam - nearest hotel: No hotel found
timimoun - nearest hotel: أناف
tharad - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
aswan - nearest hotel: Oscar Hotel
bhadra - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
dahlonega - nearest hotel: Quality Inn Dahlonega Near University
karratha - nearest hotel: Karratha International Hotel
doctor arroyo - nearest hotel: No hotel found
artesia - nearest hotel: Days Inn & Suites by Wyndham Artesia
tando muhammad khan - nearest hotel: No hotel found
laguna - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
82,afif,SA,23.9065,42.9172,52,Samaya Royal Afif
92,hoopstad,ZA,-27.8327,25.9083,12,No hotel found
97,lompoc,US,34.6391,-120.4579,47,Red Roof Inn Lompoc
125,murzuq,LY,25.9155,13.9184,31,فندق باريس - مرزق
150,port hedland,AU,-20.3167,118.5667,57,The Esplanade Hotel
206,crane,US,31.3974,-102.3501,27,No hotel found
208,guerrero negro,MX,27.9769,-114.0611,44,Plaza sal paraiso
215,santa sylvina,AR,-27.8326,-61.1375,38,No hotel found
233,aoulef,DZ,26.9667,1.0833,33,دار الضياف
234,rustam,PK,27.9690,68.8048,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [93]:
%%capture --no-display



# Configure the map plot with hover information formatted to match the example
map_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat',                            
    geo=True,                               
    tiles='OSM',                             
    size='Humidity',                        
    color='City',                            
    legend='right',                          
    frame_width=800,                         
    frame_height=500,                      
    fill_alpha=0.4,
    xlabel='x',                     
    ylabel='y',
    hover_cols=["Hotel Name", "Country"]
    
)


# Display the map

map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)